## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-20-23-52-13 +0000,nypost,Calif. Gov. Newsom signs anti-ICE bill banning...,https://nypost.com/2025/09/20/us-news/californ...
1,2025-09-20-23-48-19 +0000,nyt,Arrest in Arizona Before Charlie Kirk Memorial...,https://www.nytimes.com/2025/09/20/us/charlie-...
2,2025-09-20-23-42-52 +0000,nyt,Why Russia Revived a Cold War-Era Song Contest,https://www.nytimes.com/2025/09/20/arts/music/...
3,2025-09-20-23-28-44 +0000,bbc,"Charlie Kirk, the conservative influencer who ...",https://www.bbc.com/news/articles/cdxqnkwerj7o...
4,2025-09-20-23-25-28 +0000,nyt,"With Ouster, Justice Dept. Independence Teeter...",https://www.nytimes.com/2025/09/20/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,trump,40
14,kirk,18
13,charlie,14
270,new,12
347,kimmel,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
202,2025-09-20-00-32-00 +0000,wsj,A top federal prosecutor who has been under pr...,https://www.wsj.com/politics/policy/trump-call...,110
194,2025-09-20-01-00-00 +0000,wsj,Charlie Kirk’s widow Erika Kirk will become th...,https://www.wsj.com/us-news/erika-kirk-charlie...,99
47,2025-09-20-19-50-05 +0000,nypost,White House rips Barack Obama for suggesting T...,https://nypost.com/2025/09/20/us-news/white-ho...,88
147,2025-09-20-09-30-00 +0000,wsj,President Trump’s speech to the U.N. General A...,https://www.wsj.com/politics/policy/trumps-u-n...,78
206,2025-09-20-00-11-30 +0000,latimes,"Lopez: If he ever gets his job back, I have ju...",https://www.latimes.com/california/story/2025-...,76


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
202,110,2025-09-20-00-32-00 +0000,wsj,A top federal prosecutor who has been under pr...,https://www.wsj.com/politics/policy/trump-call...
194,87,2025-09-20-01-00-00 +0000,wsj,Charlie Kirk’s widow Erika Kirk will become th...,https://www.wsj.com/us-news/erika-kirk-charlie...
45,50,2025-09-20-20-04-18 +0000,nypost,White House tries easing panic over high-skill...,https://nypost.com/2025/09/20/us-news/white-ho...
47,35,2025-09-20-19-50-05 +0000,nypost,White House rips Barack Obama for suggesting T...,https://nypost.com/2025/09/20/us-news/white-ho...
23,35,2025-09-20-22-11-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
198,35,2025-09-20-00-35-15 +0000,nypost,"Dem mayor, anti-ICE protesters hit with tear g...",https://nypost.com/2025/09/19/us-news/anti-ice...
28,30,2025-09-20-21-33-56 +0000,nypost,High-speed police chase ends in deadly shootou...,https://nypost.com/2025/09/20/us-news/massive-...
131,29,2025-09-20-11-00-00 +0000,startribune,‘It should be called Trauma Street’: Lake Stre...,https://www.startribune.com/lake-street-shooti...
99,29,2025-09-20-15-22-28 +0000,nypost,Q&A: NYC Mayor Eric Adams calls Jews his secre...,https://nypost.com/2025/09/20/us-news/q-amp-a-...
160,29,2025-09-20-08-21-49 +0000,nypost,"Russia launches large-scale attack on Ukraine,...",https://nypost.com/2025/09/20/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
